In [1]:
import numpy as np
import pandas as pd

path = 'data/chamber_of_deputies_votes.csv'
dtype = {
    'congressperson_id': np.str,
    'session_number': np.str,
}
votes = pd.read_csv(path, dtype=dtype)
votes = votes \
    .query('term == 2015') \
    .query('legislative_session_number == 3') \
    .query('body == "chamber_of_deputies"')
votes = votes.drop_duplicates()
print(votes.shape)
votes.head()

(235, 17)


,abstention_votes,blank_votes,ending_time,first_president,name,no_votes,obstruction_votes,president_votes,term,vote_number,votes,yes_votes,body,legislative_schedule,legislative_session_number,schedule,session_number
1,4,0,2017-12-06 01:31:27,RODRIGO MAIA,MPV Nº 795/2017 - DTQ 11 - PV - ART. 7º DO TE...,222,19,1,2015,8100,369,123,chamber_of_deputies,ordinary_session,3,special_session,378
3,1,0,2017-05-16 15:25:20,JHC,MPV Nº 756/2016 - REQUERIMENTO DE RETIRADA DE...,255,61,1,2015,7538,336,18,chamber_of_deputies,ordinary_session,3,special_session,118
5,1,0,2017-03-28 20:35:55,CARLOS MANATO,MPV Nº 751/2016 - DTQ 1: PT - EMENDA Nº 34,179,0,1,2015,7445,426,245,chamber_of_deputies,ordinary_session,3,ordinary_session,049
6,2,0,2017-05-31 20:15:09,RODRIGO MAIA,PLP Nº 54/2015 - SUBSTITUTIVO DA CFT,28,5,1,2015,7594,441,405,chamber_of_deputies,ordinary_session,3,special_session,141
8,4,0,2017-02-07 18:17:32,RODRIGO MAIA,REQ Nº 3970/2016 - URGÊNCIA PARA APRECIAÇÃO D...,17,0,1,2015,7363,336,314,chamber_of_deputies,ordinary_session,3,special_session,003


In [2]:
path = 'data/chamber_of_deputies_votes_congresspeople.csv'
dtype = {
    'congressperson_id': np.str,
    'session_number': np.str,
}
votes_congresspeople = pd.read_csv(path, dtype=dtype)
votes_congresspeople = votes_congresspeople \
    .query('term == 2015') \
    .query('legislative_session_number == 3') \
    .query('body == "chamber_of_deputies"')
votes_congresspeople = votes_congresspeople.drop_duplicates()
print(votes_congresspeople.shape)
votes_congresspeople.head()

(85275, 11)


,name,vote,party,state,congressperson_id,term,body,legislative_schedule,legislative_session_number,schedule,session_number
513,CARLOS ANDRADE,no,PHS,Roraima,3,2015,chamber_of_deputies,ordinary_session,3,special_session,064
514,HIRAN GONÇALVES,no,PP,Roraima,4,2015,chamber_of_deputies,ordinary_session,3,special_session,064
515,SHÉRIDAN,no,PSDB,Roraima,8,2015,chamber_of_deputies,ordinary_session,3,special_session,064
516,ANDRÉ ABDON,no,PP,Amapá,9,2015,chamber_of_deputies,ordinary_session,3,special_session,064
517,CABUÇU BORGES,no,PMDB,Amapá,14,2015,chamber_of_deputies,ordinary_session,3,special_session,064


In [3]:
votes['acceptance_rate'] = votes['yes_votes'] / votes['votes']

In [4]:
vote_cols = [col for col in votes.columns if col[-6:] == '_votes']

def outcome(row):
    row = row[vote_cols].to_dict()
    return max(row, key=row.get)[:-6]

votes['outcome'] = votes.apply(outcome, axis=1)

In [5]:
cols = ['name', 'votes', 'schedule', 'session_number', 'acceptance_rate']

votes \
    .query('outcome == "yes"') \
    .sort_values('acceptance_rate', ascending=False)[cols].head()

,name,votes,schedule,session_number,acceptance_rate
113,PEC Nº 212/2016 - PROPOSTA DE EMENDA À CONSTI...,391,special_session,381,0.997442
172,PLP Nº 171/2015 - SUBSTITUTIVO DA CFT,334,special_session,381,0.994012
215,PLP Nº 163/2015 - PROJETO DE LEI COMPLEMENTAR,405,ordinary_session,004,0.992593
476,REQ Nº 7701/2017 - URGÊNCIA PARA APRECIAÇÃO D...,295,special_session,379,0.989831
464,PEC Nº 212/2016 - SUBSTITUTIVO DA COMISSÃO ES...,368,special_session,366,0.989130


In [6]:
keys = [
    'session_number',
    'body',
    'legislative_schedule',
    'schedule',
    'term',
    'legislative_session_number',
]

votes_congresspeople = votes_congresspeople \
    .merge(votes, on=keys, how='left', suffixes=('', '_vote'))
votes_congresspeople.iloc[0]

name                                                             CARLOS ANDRADE
vote                                                                         no
party                                                                       PHS
state                                                                   Roraima
congressperson_id                                                             3
term                                                                       2015
body                                                        chamber_of_deputies
legislative_schedule                                           ordinary_session
legislative_session_number                                                    3
schedule                                                        special_session
session_number                                                              064
abstention_votes                                                              3
blank_votes                             

In [7]:
def agg(data):
    times_right = data['vote'] == data['outcome']
    return pd.Series({
        'times_right': times_right.sum(),
    })

votes_congresspeople \
    .groupby(['name']) \
    .apply(agg) \
    .sort_values('times_right', ascending=False)

,times_right
name,
TENENTE LÚCIO,218
MARCIO ALVINO,217
CARLOS HENRIQUE GAGUIM,217
DELEGADO EDSON MOREIRA,212
HERCULANO PASSOS,211
JONES MARTINS,211
EDUARDO BARBOSA,210
ALBERTO FRAGA,210
ROBERTO ALVES,210
